# Task 1 任务分析 

1. 使用pandas读入训练集与测试集，并将文本使用词袋模型转为可供进行计算的向量集  
2. 使用numpy实现整个 `losgistic` 回归的计算过程  
3. 使用numpy实现 loss 函数、梯度下降法
4. 创建词袋向量过程中选择不同的 `N-gram`   
5. 验证 ：
    - 学习率 $\alpha$
    - 损失函数 $ loss $
    - 数据特征    
  
  对最终结果的影响
6. 对训练集/验证集/测试集进行划分

# Task 1 任务实现

## 使用 pandas 读入数据

In [4]:
import pandas as pd       
# 读入训练集
train = pd.read_csv("C:/Users/WYX/Desktop/test/train.tsv", header=0, delimiter="\t", quoting=3)
# 查看训练集大小
print('训练集大小为：\n',train.shape)
# 查看训练集的属性
print('\n训练集属性为：')
train.columns.values

训练集大小为：
 (10026, 4)

训练集属性为：


array(['ID1', 'ID2', 'Sentiment', 'Sentence'], dtype=object)

注：在做机器学习的任务时，需要在运行模型之前将特征转化成词 id 再转化成模型可识别的二进制文件形式，其中转化成的词 `id` 文件最好进行 `shuffle`，打乱各行数据，这样参数能不易陷入局部最优，模型能够更容易达到收敛。

In [56]:
# # 对训练集进行 shuffle 操作
from sklearn.utils import shuffle
print('Shuffle 前：\n')
print(train["Phrase"][0])
print('\n')
print(train["Phrase"][1])
train = train.sample(frac=1)
train= train.reset_index(drop=True)
print('\nShuffle 后：\n')
print(train["Phrase"][0])
print('\n')
print(train["Phrase"][1])

Shuffle 前：

A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .


A series of escapades demonstrating the adage that what is good for the goose

Shuffle 后：

got me grinning .


a risky venture that never quite goes where you expect and often surprises you with unexpected comedy


注：这里 `train= train.reset_index(drop=True)` 这一行代码一定要加上，因为我们访问 `DataFrame` 是使用索引进行访问的，所以进行重新排序之后一定要进行索引的更改 

## 对读入训练集数据进行预处理

In [ ]:
import nltk
import re
from bs4 import BeautifulSoup 
from nltk.corpus import stopwords
# 定义文本处理函数
def review_to_words( raw_review ):
    # 去除额外标签
    review_text = BeautifulSoup(raw_review).get_text()       
    # 使用正则表达式去除数字 与非字母符号
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    # 将单词转为小写之后划分为 list 列表
    words = letters_only.lower().split()                             
    # 去除停用词
    stops = set(stopwords.words("english"))                  
    # 获得单个词列表
    meaningful_words = [w for w in words if not w in stops]   
    return( " ".join( meaningful_words ))   

# 单个样例进行处理与效果查看
print('原句子为：\n',train["Phrase"][0])
test = review_to_words(train["Phrase"][0])
print('\n')
print('处理后的新句子为：\n',test)

# 对整个训练集进行处理
new_train = []
NumberofSize = train["Phrase"].size
for i in list(range(0,NumberofSize)):
    new_train.append(review_to_words(train["Phrase"][i]))

原句子为：
 got me grinning .


处理后的新句子为：
 got grinning


In [4]:
print('新 train 类型为：',type(new_train))
print('新 train 部分数据查看:\n')
print('No.1:',new_train[0])
print('No.2:',new_train[1])
print('No.3:',new_train[2])

新 train 类型为： <class 'list'>
新 train 部分数据查看:

No.1: grubbing
No.2: often heartbreaking testimony spoken directly director patricio guzman camera pack powerful emotional wallop
No.3: durable best seller smart women


## 使用词袋模型完成从文本到向量的转换


In [6]:
# 导入向量化工具 CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(ngram_range=(1,1),max_features = 5000)

注：
由于在转换成词袋模型的过程中在 n-gram = 1 的情况下由于不同单词的种类为 14903 所以在notebook中构建词向量过程中会出现内存错误的情况，为了解决这个问题，在查询资料之后采用 kaggle 竞赛教程的方法：**选择在创建词袋向量时候规定只选用出现词频最高的 5000 个单词进行词袋向量的构建**

In [8]:
# 拟合文本数据 建立词袋模型
vect.fit(new_train)

#  得到参数
print('\n得到参数: ',vect.get_params())

#  输出特征数
print('\n单词数量: ',len(vect.vocabulary_))

#  得到特征的名字
print('\n向量特征的名字：',vect.get_feature_names())

#  得到分词
print('\n分词: ',vect.vocabulary_)

#  得到对应的词袋模型
print('\n词袋向量: ')
print(vect.fit_transform(new_train).toarray().shape)
X_data = vect.fit_transform(new_train).toarray() 


得到参数:  {'analyzer': 'word', 'binary': False, 'decode_error': 'strict', 'dtype': <class 'numpy.int64'>, 'encoding': 'utf-8', 'input': 'content', 'lowercase': True, 'max_df': 1.0, 'max_features': 5000, 'min_df': 1, 'ngram_range': (1, 1), 'preprocessor': None, 'stop_words': None, 'strip_accents': None, 'token_pattern': '(?u)\\b\\w\\w+\\b', 'tokenizer': None, 'vocabulary': None}

单词数量:  5000

向量特征的名字： ['abandon', 'ability', 'able', 'absolute', 'absolutely', 'absorbing', 'abstract', 'absurd', 'absurdity', 'abundant', 'abuse', 'academy', 'accent', 'accents', 'accept', 'acceptable', 'access', 'accessible', 'accident', 'accomplished', 'accomplishments', 'account', 'accurate', 'accurately', 'acerbic', 'ache', 'achieve', 'achievement', 'achieves', 'achingly', 'acknowledges', 'across', 'act', 'acted', 'acting', 'action', 'actions', 'actor', 'actors', 'actress', 'actresses', 'acts', 'actual', 'actually', 'ad', 'adam', 'adams', 'adaptation', 'adapted', 'add', 'added', 'addition', 'adds', 'adequate

(156060, 5000)


从上一步的输出中可以发现得到的 `vect.fit_transform(new_train).toarray()` 是一个 numpy 类型的数据(narray),所以在后面的运算过程中可以直接进行操作 

# 基于 softmax 实现多元 logistic 回归多分类问题

## logistic 回归原理

参考链接为：http://ufldl.stanford.edu/wiki/index.php/Softmax%E5%9B%9E%E5%BD%92

In [5]:
# 定义 softmax 类并根据传入数据集设置参数矩阵 w 的大小
def __init__(self, maxstep=1000, C=1e-4, alpha=0.4):
        # 设置最大迭代次数
        self.maxstep = maxstep
        # 设置乘法系数
        self.C = C 
        # 设置学习率
        self.alpha = alpha
        # 设置权值 w 矩阵
        self.w = None  
        # 设置待分类的数量
        self.L = None 
        # 设置数据维数
        self.D = None  # 输入数据维度
        # 设置样本行数
        self.N = None  # 样本总量
def init_param(self, X_data, y_data):
        # 设置常数项 b 保证运算形式为 Wx + b = y
        b = np.ones((X_data.shape[0], 1))
        X_data = np.hstack((X_data, b))
        # 统计待分类的种类数
        self.L = len(np.unique(y_data))
        # 设置 w 矩阵宽度
        self.D = X_data.shape[1]
        # 设置 w 矩阵长度
        self.N = X_data.shape[0]
        self.w = np.ones((self.L, self.D))  # l*d, 针对每个类，都有一组权值参数w
        return X_data

## 代价函数

$\begin{aligned} J(\theta) &=-\frac{1}{m}\left[\sum_{i=1}^{m}\left(1-y^{(i)}\right) \log \left(1-h_{\theta}\left(x^{(i)}\right)\right)+y^{(i)} \log h_{\theta}\left(x^{(i)}\right)\right] \\ &=-\frac{1}{m}\left[\sum_{i=1}^{m} \sum_{j=0}^{1} 1\left\{y^{(i)}=j\right\} \log p\left(y^{(i)}=j | x^{(i)} ; \theta\right)\right] \end{aligned}$

注意在Softmax回归中将 $x$ 分类为类别 $j$ 的概率为：  

$p\left(y^{(i)}=j | x^{(i)} ; \theta\right)=\frac{e^{\theta_{j}^{T} x^{(i)}}}{\sum_{l=1}^{k} e^{\theta_{l}^{T} x^{(i)}}}$

对代价函数经过求导，我们得到梯度公式如下：   

$\nabla_{\theta_{j}} J(\theta)=-\frac{1}{m} \sum_{i=1}^{m}\left[x^{(i)}\left(1\left\{y^{(i)}=j\right\}-p\left(y^{(i)}=j | x^{(i)} ; \theta\right)\right)\right]$


得到梯度之后，随后在每一次迭代中进行如下更新：  
$\theta_{j} :=\theta_{j}-\alpha \nabla_{\theta_{j}} J(\theta)(j=1,....,k)$

## 参数衰减

通过添加一个权重衰减项 $ \textstyle \frac{\lambda}{2} \sum_{i=1}^k \sum_{j=0}^{n} \theta_{ij}^2$ 来修改代价函数，这个衰减项会惩罚过大的参数值，代价函数变为：  
$J(\theta)=-\frac{1}{m}\left[\sum_{i=1}^{m} \sum_{j=1}^{k} 1\left\{y^{(i)}=j\right\} \log \frac{e^{\theta T_{x}(i)}}{\sum_{l=1}^{k} e^{\theta_{l}^{T} x^{(i)}}}\right]+\frac{\lambda}{2} \sum_{i=1}^{k} \sum_{j=0}^{n} \theta_{i j}^{2}$

有了这个权重衰减项以后 $(\textstyle \lambda > 0)$，代价函数就变成了严格的凸函数，这样就可以保证得到唯一的解了。

新 $J(\theta)$ 的导数为：  

$\nabla_{\theta_{j}} J(\theta)=-\frac{1}{m} \sum_{i=1}^{m}\left[x^{(i)}\left(1\left\{y^{(i)}=j\right\}-p\left(y^{(i)}=j | x^{(i)} ; \theta\right)\right)\right]+\lambda \theta_{j}$

In [ ]:
grad = -1.0 / self.N * prob.T @ X_data + self.C * self.w

## 梯度下降法

使用梯度下降法完成对 w 权值矩阵的求解，参考公式为：  
$\theta_{j} :=\theta_{j}-\alpha \nabla_{\theta_{j}} J(\theta)(j=1,....,k)$

In [6]:
def bgd(self, X_data, y_data):
        # 梯度下降训练
        step = 0
        while step < self.maxstep:
            step += 1
            if step % 100 == 0:
                print('now step is :',step);
            # 计算当前迭代下的值
            prob = np.exp(X_data @ self.w.T) 
            nf = np.transpose([prob.sum(axis=1)])  
            nf = np.repeat(nf, self.L, axis=1)  
            # 归一化处理
            prob = -prob / nf  
            for i in range(self.N):
                prob[i, int(y_data[i])] += 1
            # 梯度下降
            grad = -1.0 / self.N * prob.T @ X_data + self.C * self.w  # 梯度， 第二项为衰减项
            self.w -= self.alpha * grad
        return

## logistic 回归 与 softmax 回归的关系

当待分类的类别为 k = 2时，softmax 退化为 logistic，也即 softmax 回归是 logistic 回归的一般形式；

# 程序实现代码综合

In [ ]:
import pandas as pd       

train = pd.read_csv("C:/Users/WYX/Desktop/mytest/train.tsv", header=0, delimiter="\t", quoting=3)


train.columns.values


train = train.sample(frac=1)
train= train.reset_index(drop=True)


import nltk
import re
from bs4 import BeautifulSoup 
from nltk.corpus import stopwords

def review_to_words( raw_review ):

    review_text = BeautifulSoup(raw_review).get_text()       

    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    words = letters_only.lower().split()                             

    stops = set(stopwords.words("english"))                  

    meaningful_words = [w for w in words if not w in stops]   
    return( " ".join( meaningful_words ))


new_train = []
NumberofSize = train["Phrase"].size
for i in list(range(0,NumberofSize)):
    new_train.append(review_to_words(train["Phrase"][i]))

from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(ngram_range=(1,1),max_features = 5000)

vect.fit(new_train)
X_data = vect.fit_transform(new_train).toarray()

import numpy as np


class SoftMax:
    def __init__(self, maxstep=10000, C=1e-4, alpha=0.4):
        self.maxstep = maxstep
        self.C = C  # 权值衰减项系数lambda, 类似于惩罚系数
        self.alpha = alpha  # 学习率

        self.w = None  # 权值

        self.L = None  # 类的数量
        self.D = None  # 输入数据维度
        self.N = None  # 样本总量

    def init_param(self, X_data, y_data):
        # 初始化，暂定输入数据全部为数值形式
        b = np.ones((X_data.shape[0], 1))
        X_data = np.hstack((X_data, b))  # 附加偏置项
        self.L = len(np.unique(y_data))
        self.D = X_data.shape[1]
        self.N = X_data.shape[0]
        self.w = np.ones((self.L, self.D))  # l*d, 针对每个类，都有一组权值参数w
        return X_data

    def bgd(self, X_data, y_data):
        # 梯度下降训练
        step = 0
        while step < self.maxstep:
            step += 1
            prob = np.exp(X_data @ self.w.T)  # n*l, 行向量存储该样本属于每个类的概率
            nf = np.transpose([prob.sum(axis=1)])  # n*1
            nf = np.repeat(nf, self.L, axis=1)  # n*l
            prob = -prob / nf  # 归一化， 此处条件符号仅方便后续计算梯度
            for i in range(self.N):
                prob[i, int(y_data[i])] += 1
            grad = -1.0 / self.N * prob.T @ X_data + self.C * self.w  # 梯度， 第二项为衰减项
            self.w -= self.alpha * grad
        return

    def fit(self, X_data, y_data):
        X_data = self.init_param(X_data, y_data)
        self.bgd(X_data, y_data)
        return

    def predict(self, X):
        b = np.ones((X.shape[0], 1))
        X = np.hstack((X, b))  # 附加偏置项
        prob = np.exp(X @ self.w.T)
        return np.argmax(prob, axis=1)


if __name__ == '__main__':

    y_data = []
    NumberofSize = train["Sentiment"].size
    for i in list(range(0,NumberofSize)):
        y_data.append(train["Sentiment"][i])

    from sklearn.model_selection import train_test_split

    X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=1)
    clf = SoftMax(maxstep=1000, alpha=0.1, C=1e-4)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    score = 0
    for y, y_pred in zip(y_test, y_pred):
        score += 1 if y == y_pred else 0
    print(score / len(y_test))

